In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

# 기본 import

In [2]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.prompts.chat import ChatPromptTemplate

# Memory

https://python.langchain.com/api_reference/langchain/memory.html

챗봇으로 하여금 대화(상태)를 '기억'하게끔 한다

Memory maintains Chain state, incorporating context from past runs.


In [ ]:
# Langchain 의 memory 계층
#  BaseMemory --> BaseChatMemory --> <name>Memory  # Examples: ZepMemory, MotorheadMemory

In [ ]:
# OpenAI 사에서 제공하는 '기본 API' 도 랭체인 없이 사용 가능.
# 메모리 지원하지 않는다.  이전 대화 기억 못함.  stateless 하다!

# ChatGPT 서비스 는 '메모리' 기능이 탑재되어 있다.
# 챗봇이 이전의 대화 내용과 질문을 기억하고 답할수 있다.


# 1.ConversationBufferMemory

In [3]:
# ConversationBufferMemory
# 대화 내용 '전체'를 저장하는 메모리

# 장점: 단순하다

# 단점:
# => 매번 요청할때마다 '이전 대화 기록 전체' 를 같이 보내야 함.
#  그래야 모델이 전에 일어났던 대화를 보고 이해 할수 있다.
#  대화내용이 길어질수록 메모리도 계속 커지니까 성능적으로도 & 비용적으로도 비효율적이다.


In [4]:
from langchain.memory.buffer import ConversationBufferMemory

In [6]:
memory = ConversationBufferMemory()

# 직접 save
memory.save_context(
    {'input': 'Hi!'},   # 유저 입력값
    {'output': 'How are you?'}, # AI 가 유저에게 답한 내용 기록
)

memory.load_memory_variables({})   # inputs= 매개변수가 빈dict


# {'history': 'Human: Hi!\nAI: How are you?'}
#  history 가 텍스트다


{'history': 'Human: Hi!\nAI: How are you?'}

## return_messages=True
history 에  AIMessage 와 HumanMessage 로 저장된다.

In [7]:
memory = ConversationBufferMemory(return_messages=True)

memory

ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[]), return_messages=True)

In [8]:
memory.save_context(
    {'input': 'Hi!'},   # 유저 입력값
    {'output': 'How are you?'}, # AI 가 유저에게 답한 내용 기록
)

memory.load_memory_variables({})

# {'history': [HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}),
#   AIMessage(content='How are you?', additional_kwargs={}, response_metadata={})]}

#  history 가 Message 의 list 다!

{'history': [HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}),
  AIMessage(content='How are you?', additional_kwargs={}, response_metadata={})]}

In [10]:
memory.save_context(
    {'input': 'Hi!'},   # 유저 입력값
    {'output': 'How are you?'}, # AI 가 유저에게 답한 내용 기록
)

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}),
  AIMessage(content='How are you?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}),
  AIMessage(content='How are you?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}),
  AIMessage(content='How are you?', additional_kwargs={}, response_metadata={})]}

# 2.ConversationBufferWindowMemory

In [11]:
from langchain.memory.buffer_window import ConversationBufferWindowMemory

In [ ]:
# ConversationBufferWindowMemory 는 대화의 '특정 부분만' 을 저장하는 메모리.

# 장점:
#   메모리를 특정 크기로 유지할 수 있다!
#   따라서 모든 대화 내용을 저장하지 않아도 된다!

# 단점:
#   챗봇이 전체 대화가 아닌 '최근 대화' 에만 집중하게 된다.


In [12]:
memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4,   # 버퍼 윈도우 사이즈.  몇개의 context(메세지)를 저장할지 지정
)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14796\823005658.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(


In [13]:
# 도우미 함수 준비.
def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

In [14]:
# 동작 확인
add_message("1", "1")
add_message("2", "2")
add_message("3", "3")
add_message("4", "4")

In [15]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='1', additional_kwargs={}, response_metadata={}),
  AIMessage(content='1', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='2', additional_kwargs={}, response_metadata={}),
  AIMessage(content='2', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='3', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='4', additional_kwargs={}, response_metadata={}),
  AIMessage(content='4', additional_kwargs={}, response_metadata={})]}

In [16]:
add_message("5", "5")
memory.load_memory_variables({})

{'history': [HumanMessage(content='2', additional_kwargs={}, response_metadata={}),
  AIMessage(content='2', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='3', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='4', additional_kwargs={}, response_metadata={}),
  AIMessage(content='4', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='5', additional_kwargs={}, response_metadata={}),
  AIMessage(content='5', additional_kwargs={}, response_metadata={})]}

# 3.ConversationSummaryMemory

In [ ]:
# ConversationSummaryMemory 는 LLM 을 사용하여 대화의 요약본 (summary) 생성

In [17]:
llm = ChatOpenAI(temperature=0.1)

In [18]:
from langchain.memory.summary import ConversationSummaryMemory

In [19]:
# ConversationSummaryMemory
# 메세지를 그대로 저장하는 것이 아니라 Convertaion 의 '요약'을 해준다. LLM 필요

# 장점: 대화의 메세지가 많아질수록 요약을 해주어 토큰의 양도 줄어들어 훨씬 경제적인 방법
# 단점: LLM 호출 발생

memory = ConversationSummaryMemory(llm=llm)  # llm 을 사용하여 요약한다!



C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14796\2664529100.py:7: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(llm=llm)  # llm 을 사용하여 요약한다!


In [20]:
def get_history():
    return memory.load_memory_variables({})

In [21]:
# message 추가
add_message(
    "Hi I'm John, I live in South Korea",    # input
    "Wow that is so cool!"  # output : AI 답변
    )

In [22]:
# 또 message 추가
add_message(
    "South Korea is so pretty",
    "I wish I could go!!!")

In [23]:
get_history()

{'history': 'John introduces himself as living in South Korea. The AI responds by expressing admiration for his location, wishing it could go there.'}

# 4.ConversationSummaryBufferMemory

In [24]:
from langchain.memory.summary_buffer import ConversationSummaryBufferMemory

In [25]:
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=150,   # 최대 가용한 메세지 토큰수 (메세지들이 요약되지 전)
    return_messages=True,
)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14796\2068819690.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryBufferMemory(


In [26]:
add_message(
    "Hi I'm John, I live in South Korea",    # input
    "Wow that is so cool!"  # output : AI 답변
    )
get_history()

{'history': [HumanMessage(content="Hi I'm John, I live in South Korea", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Wow that is so cool!', additional_kwargs={}, response_metadata={})]}

In [27]:
# 다시 메세지 추가하고 history 확인
add_message(
    "South Korea is so pretty",
    "I wish I could go!!!")
get_history()

{'history': [HumanMessage(content="Hi I'm John, I live in South Korea", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Wow that is so cool!', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='South Korea is so pretty', additional_kwargs={}, response_metadata={}),
  AIMessage(content='I wish I could go!!!', additional_kwargs={}, response_metadata={})]}

In [28]:
add_message(
    "How far is Korea from Argentina?",
    "I don't know! Super far!"
)
get_history()

{'history': [HumanMessage(content="Hi I'm John, I live in South Korea", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Wow that is so cool!', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='South Korea is so pretty', additional_kwargs={}, response_metadata={}),
  AIMessage(content='I wish I could go!!!', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='How far is Korea from Argentina?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I don't know! Super far!", additional_kwargs={}, response_metadata={})]}

In [32]:
# 아래 셀을 여러차레 해보자  약 (3,4번?)
# =>실제 '요약'이 발생할때면 Model IO 가 발생하기 때문에 시간이 좀 걸리는게 느껴질거다!
add_message(
    "How far is Brazil from Argentina?",
    "I don't know! Super far!"
)
get_history()

{'history': [SystemMessage(content='The human introduces himself as John and mentions that he lives in South Korea.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Wow that is so cool!', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='South Korea is so pretty', additional_kwargs={}, response_metadata={}),
  AIMessage(content='I wish I could go!!!', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='How far is Korea from Argentina?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I don't know! Super far!", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='How far is Brazil from Argentina?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I don't know! Super far!", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='How far is Brazil from Argentina?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I don't know! Super far!", additional_kwar

In [ ]:
# max_token_limit 에 도달하면, 오래된 메세지들이 요약되고 있을 것을 확인할수 있다. (SystemMessage 확인)

# ★그러나 '요약' 이라는 과정은 API 를 사용한다는 사실을 명심하세요.
# ★'요약' 동작은 비용 지출이 발생되는 부분입니다.


# 5.ConversationKGMemory
KG : Knowledge Graph

In [33]:
# 대화중에 '엔티티'의 knowledge graph 를 형성한다 => 가장 중요한 것들만 추출한 요약본.
# knowledge graph 는 history 를 가지고 오지 않는다.  대신 '엔티티' 를 가지고 옴


In [34]:
from langchain_community.memory.kg import ConversationKGMemory

In [35]:
memory = ConversationKGMemory(
    llm=llm,  # 이 또한 LLM 을 사용하는 Memory 다  -> Knowledge Graph 를 만든다. (가장 중요한것만 뽑아낸 요약본)
    return_messages=True,
)

In [36]:
add_message(
    "Hi I'm John, I live in South Korea",    # input
    "Wow that is so cool!"  # output : AI 답변
    )

In [37]:
# 대화의 특정 entity 에 대해 질무ㄴ해보자
#  Model IO 발생
memory.load_memory_variables({"input": "Who is John"})   # 빈dict 가 아니다!

{'history': [SystemMessage(content='On John: John lives in South Korea.', additional_kwargs={}, response_metadata={})]}

In [38]:
add_message("John likes kimchi", "Wow that is so cool!")

In [39]:
memory.load_memory_variables({"input": "Who is John"})

{'history': [SystemMessage(content='On John: John lives in South Korea. John likes kimchi.', additional_kwargs={}, response_metadata={})]}

# 6. 참고: Database 와 integration 된 메모리들

- **참고: Database 와 integration 된 메모리들**

| Memory Class                  | 통합 대상 (Integration)             | 설명                                                           |
| ----------------------------- | ------------------------------- | ------------------------------------------------------------ |
| `RedisChatMessageHistory`     | **Redis**                       | Redis에 메시지 저장. 빠르고 확장 가능한 저장소.                               |
| `SQLChatMessageHistory`       | **SQLite, PostgreSQL 등 SQL DB** | SQL 데이터베이스에 메시지 저장. SQLAlchemy 기반.                           |
| `DynamoDBChatMessageHistory`  | **AWS DynamoDB**                | AWS의 NoSQL DB인 DynamoDB에 대화 저장. 서버리스 환경에서 유용.                |
| `MongoDBChatMessageHistory`   | **MongoDB**                     | 문서 기반 DB인 MongoDB와 통합하여 대화 저장.                               |
| `PostgresChatMessageHistory`  | **PostgreSQL**                  | PostgreSQL 전용 구현 (SQLAlchemy 없이).                            |
| `FileChatMessageHistory`      | **Local 파일**                    | JSON 파일로 로컬에 저장. 간단한 로깅에 적합.                                 |
| `FirestoreChatMessageHistory` | **Google Firestore**            | Firebase 기반 클라우드 NoSQL DB와 통합.                               |
| `ChromaMemory`                | **Chroma (벡터 DB)**              | 벡터 DB에 embedding 형태로 memory 저장. RAG나 유사 검색 기반 memory에 활용 가능. |
| `WeaviateMemory`              | **Weaviate**                    | Weaviate 벡터 DB와 통합된 memory 저장.                               |
| `QdrantMemory`                | **Qdrant**                      | 벡터 기반 memory 저장소로 Qdrant 사용.                                 |


# 7.Memory on LLMChain

In [40]:
# 메모리를 'chain' 에 꽂는 방법.
# 두가지 형태의 'chain' 에 각각 꽂는 방법.
#    1. off-the-shelf chain  : LangChain 에서 '일반적인 목적'을 수행하는 기본 제공되는 chain
#    2. LCEL 사용한 chain : 커스텀 chains


In [41]:
from langchain.chains.llm import LLMChain

In [42]:
llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm, 
    max_token_limit=80,
)

chain = LLMChain(   # prompt | llm
    llm=llm,
    memory=memory,  # 메모리 제공
    prompt=PromptTemplate.from_template("{question}")
)



C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14796\2707029833.py:8: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(   # prompt | llm


In [43]:
chain.invoke(input={"question": "My name is John"})

{'question': 'My name is John',
 'history': '',
 'text': 'Nice to meet you, John! How can I assist you today?'}

In [44]:
chain.invoke(input={"question": "I live in Seoul"})

{'question': 'I live in Seoul',
 'history': 'Human: My name is John\nAI: Nice to meet you, John! How can I assist you today?',
 'text': ', the capital city of South Korea. It is a vibrant and bustling metropolis with a rich history and culture. There are so many things to see and do in Seoul, from visiting ancient palaces and temples to exploring modern shopping districts and enjoying delicious Korean cuisine. I love living in Seoul because of its dynamic energy and the endless opportunities for adventure and exploration.'}

In [45]:
# 과연 이름을 기억하고 있을까?
chain.invoke(input={'question':'What is my name?'})

{'question': 'What is my name?',
 'history': 'System: The human introduces himself as John. The AI greets John and asks how it can assist him. John mentions that he lives in Seoul.\nAI: , the capital city of South Korea. It is a vibrant and bustling metropolis with a rich history and culture. There are so many things to see and do in Seoul, from visiting ancient palaces and temples to exploring modern shopping districts and enjoying delicious Korean cuisine. I love living in Seoul because of its dynamic energy and the endless opportunities for adventure and exploration.',
 'text': "I'm sorry, I do not know your name as I am an AI assistant and do not have access to personal information."}

In [46]:
# 이름을 모른다고???

# chain 을 디버깅해보자.

## verbose=
chain 을 실행했을때 chain 의 프롬프트와 로그들을 확인할수 있다. (디버깅용)

In [47]:
chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template("{question}"),
    verbose=True,  # chain 을 실행했을때 chain 의 프롬프트 로그들을 확인할수 있다. (디버깅용)
)

chain.invoke(input={'question':"My name is John"})
chain.invoke(input={'question':"I live in Seoul"})
chain.invoke(input={'question':"What is my name?"})



> Entering new LLMChain chain...
Prompt after formatting:
My name is John

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
I live in Seoul

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
What is my name?

> Finished chain.


{'question': 'What is my name?',
 'history': "System: The human, named John, mentions that he lives in Seoul. The AI responds by acknowledging John's name and location, expressing pleasure in meeting him and offering assistance.\nAI: , the capital city of South Korea. It is a bustling metropolis with a vibrant culture, delicious food, and a mix of modern skyscrapers and historic palaces. I love exploring the city's neighborhoods, trying new restaurants, and taking in the beautiful views from Namsan Mountain. Seoul is a dynamic and exciting place to call home.",
 'text': "I'm sorry, I do not have access to personal information such as your name."}

In [ ]:
# ↑ chain 의 prompt 로그 들을 확인할 수 있다.
# 보다시피 대화의 내역(history) 가 prompt에 계속 추가되진 않는다.

# 우리가 원하는 어떤 방식으로 prompt에게 대화 기록(history)을 추가해줘야 한다!

# ↓ 하지만! 메모리는 계속 업데이트 된다.  함 보자!

In [48]:
memory.load_memory_variables({})

{'history': "System: John mentions that he lives in Seoul, the capital city of South Korea. The AI responds by acknowledging John's name and location, expressing pleasure in meeting him and offering assistance. The AI then shares its love for Seoul, describing it as a bustling metropolis with a vibrant culture, delicious food, and a mix of modern skyscrapers and historic palaces. The AI enjoys exploring the city's neighborhoods, trying new restaurants, and taking in the beautiful views from Namsan Mountain, considering Seoul a dynamic and exciting place to call home.\nHuman: What is my name?\nAI: I'm sorry, I do not have access to personal information such as your name."}

In [49]:
# 위 메모리 내용이 prompt 에 포함되어야 한다!

# memory_key= 에 "chat_history" 라고 말해주어야 한다.

## memory_key=

In [50]:
memory = ConversationBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history", 
)

# ↓ template 안에는 memory 가 history 를 저장하도록 한 곳을 적어주기만 하면 된다
# history 까지 담을 괜찮은 템플릿을 준비해보자
template = """
    You are a helpful AI talking to a human.

    {chat_history}
    Human:{question}
    You:
"""
# AI 가 우리의 대화기록을 기억하면서 여기의 question 을 완성할 수 있기를 기대해보자.

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),  # <- template 지정
    verbose=True,  # 어떤 prompt 가 만들어질까?  두근두근
)

In [51]:
chain.invoke({'question': "My name is John"})



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    
    Human:My name is John
    You:


> Finished chain.


{'question': 'My name is John',
 'chat_history': '',
 'text': 'Hello John! How can I assist you today?'}

In [52]:
chain.invoke({'question': "I live in Seoul"})



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    Human: My name is John
AI: Hello John! How can I assist you today?
    Human:I live in Seoul
    You:


> Finished chain.


{'question': 'I live in Seoul',
 'chat_history': 'Human: My name is John\nAI: Hello John! How can I assist you today?',
 'text': "That's great to know! How can I assist you with information or tasks related to Seoul?"}

In [53]:
chain.invoke({'question':'What is my name?'})



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    Human: My name is John
AI: Hello John! How can I assist you today?
Human: I live in Seoul
AI: That's great to know! How can I assist you with information or tasks related to Seoul?
    Human:What is my name?
    You:


> Finished chain.


{'question': 'What is my name?',
 'chat_history': "Human: My name is John\nAI: Hello John! How can I assist you today?\nHuman: I live in Seoul\nAI: That's great to know! How can I assist you with information or tasks related to Seoul?",
 'text': 'Your name is John.'}

In [54]:
# - 프롬프트 템플릿 안에서 메모리 내용이 들어갈 공간을 준비한다.  (예: chat_history)
# - 메모리를 활용할 템플릿은 원하는대로 작성하면 된다.
# - Memory 클래스에선 history 를 어디에 꽂을지 지정해준다 (memory_key=)
